### Codio Activity 23.4: Fine Tuning a Pre-trained Network

**Expected Time = 60 minutes**

**Total Points = 30**

In addition to the use of a pre-trained network to extract features from a different dataset, the weights can be adjusted or **fine-tuned** as a last step to squeeze additional performance from the network.  To do so, you will again use the `EfficientNetV2B0` network on the `cifar10` data from `keras`.  This time you are encouraged to use the functional API syntax to construct your network.  

For a second example, consult the `keras` documentation example [here](https://keras.io/guides/transfer_learning/). 

#### Index

- [Problem 1](#-Problem-1)
- [Problem 2](#-Problem-2)
- [Problem 3](#-Problem-3)

Run the code cell below to import the necessary libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Model, Input

2024-07-21 21:35:40.208242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-21 21:35:40.208316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
sub_indices = (
    pd.DataFrame(y_train, columns=["labels"])
    .groupby("labels", group_keys=False)
    .apply(lambda x: x.sample(frac=0.3, random_state=123))
    .index
)
X_train = X_train[sub_indices]
y_train = y_train[sub_indices]
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

[Back to top](#-Index)

### Problem 1

#### Training the Network to Convergence

**10 Points**

In the code cell below, use the function `EfficientNetV2B0` with the appropriate input shape and the argument `include_top` equal to `False` to create the model you will be using for this activity. Assign your result to `base_model`.

Use the function `Input()` with argument `shape` equal to `(32, 32, 3)` and assign your result to the variable `inputs`.

Use `base_model` with argument equal to `inputs` and assign your result to the variable `x`.

Use the function `Flatten` to flatten `x`. The pseudocode to complete this step is given below:

```Python
x = Flatten()(...)
```

Pass `x` through a `Dense` layer with 100 hidden nodes and `activation` equal to `relu` and assign the result to the variable `output`. The pseudocode to complete this step is given below:

```Python
output = Dense(..., activation = ...)(...)
```

Use the code `base_model.trainable = False` to ensure that your weights are not trainable.

Use the function `Model()` with argument `inputs` and `output` to define your model. Assign the result to the variable `model`.

Compile `model` using `categorical_crossentropy` as your `loss` and `accuracy` as your `metric`.

Use the `fit()` function on `model` to fit the  training data `X_train` and `Y_train`. Set the argument `validation_data` equal to `(X_test, Y_test)` and the argument `epochs` equal to 1.  Assign the result to the variable `bottom_model` below. 

NOTE: This question is computationally expensive, so please be patient with the processing. It may take a few minutes based on your computing power. 


In [42]:
### GRADED
base_model = EfficientNetV2B0(input_shape=(32, 32, 3), include_top=False)
base_model.trainable = False
inputs = Input(shape=(32, 32, 3))
x = base_model(inputs)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
output = Dense(10, activation="sigmoid")(x)
model = Model(inputs, output)
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

### ANSWER CHECK
print(model.summary())

bottom_model = model.fit(
    X_train,
    Y_train,
    epochs=1,
    verbose=0,
    validation_data=(X_test, Y_test),
)

print(bottom_model.history)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 efficientnetv2-b0 (Function  (None, 1, 1, 1280)       5919312   
 al)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 1280)              0         
                                                                 
 dense_6 (Dense)             (None, 100)               128100    
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 6,048,422
Trainable params: 129,110
Non-trainable params: 5,919,312
___________________________________________

[Back to top](#-Index)

### Problem 2

#### Setting to Trainable

**10 Points**

In the code cell below, use the code `base_model.trainable = True` to ensure that your weights are now trainable.


Set the final five layers to trainable in the `base_model` as demonstrated in the lectures. 

In [40]:
base_model.trainable  # are the base model weights set to trainable?

False

In [41]:
### GRADED
base_model.trainable = False
for layer in base_model.layers[-5:]:
    # make trainable
    layer.trainable = True

### ANSWER CHECK
for i, layer in enumerate(base_model.layers[-10:]):
    print(f"Layer[{i}] is trainable: {layer.trainable}")

Layer[0] is trainable: False
Layer[1] is trainable: False
Layer[2] is trainable: False
Layer[3] is trainable: False
Layer[4] is trainable: False
Layer[5] is trainable: True
Layer[6] is trainable: True
Layer[7] is trainable: True
Layer[8] is trainable: True
Layer[9] is trainable: True


[Back to top](#-Index)

### Problem 3

#### Refitting the network

**10 Points**

In the code cell below, use the function `compile` on `model` using `categorical_crossentropy` as your `loss` and `accuracy` as your `metric`.

Next, use the `fit()` function on `model` to fit the  training data `X_train` and `Y_train`. Set the argument `validation_data` equal to `(X_test, Y_test)` and the argument `epochs` equal to 1.  Assign the result to the variable `fine_tuned_history` below. 
 

In [ ]:
### GRADED
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

fine_tuned_history = model.fit(
    X_train,
    Y_train,
    epochs=1,
    verbose=0,
    validation_data=(X_test, Y_test),
)

### ANSWER CHECK
fine_tuned_history.history["accuracy"][-1]